In [1]:
!pip install opensearch-py

     |████████████████████████████████| 266kB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 163kB 85.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 24.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 59.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 74.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 68.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 23.8MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jsonschema-path 0.3.2 has requirement referencing<0.32.0,>=0.28.0, but you'll have referencing 0.33.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from opensearchpy import OpenSearch
host = 'vpc-aw-fdc-opensearch-qa-01-klbmt4ru7oksvsemnnmjpohfw4.us-east-1.es.amazonaws.com'
port = 443
auth = ('qa-developer', 'wF5341W8=~') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [17]:
query = {
  "size" : 1000,
  "sort" : [
        { "@timestamp" : {"order" : "desc"}}
  ],
  "query": {
    "bool": {
        "must": [
            {"match_phrase" : {"kubernetes.labels.app": "fosfor-gatekeeper"}},
            {"regexp": {"message": "[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}"}}
        ],
        "filter" : { "bool" :   {   
            "must_not": [{"regexp" : {"message": "elb"}}]
            }
        }
      }
    }
}
    
response = client.search(
    body = query,
    index = 'filebeat-qa-*'
)

#response

In [35]:
import sys

l_arr = []
for hit in response['hits']['hits']:
    s_arr = []
    str = hit["_source"]["message"].split()
    print(str)
    if str.length() > 9:
        s_arr.append(str[6])
        s_arr.append(str[8])
        s_arr.append(str[9])
        l_arr.append(s_arr)


['10.229.14.228', '-', '-', '[23/May/2024:12:19:23', '+0000]', '"GET', '/analytics-config/v1/dataset/datasetId_Not%20Found/kpi?sort=_modified_date+desc&version=draft&status=enabled%2Cdisabled%2Cissue%2Cdraft', 'HTTP/1.1"', '500', '56', '"-"', '"Mozilla/5.0', '(Windows', 'NT', '10.0;', 'Win64;', 'x64)', 'AppleWebKit/537.36', '(KHTML,', 'like', 'Gecko)', 'Chrome/124.0.0.0', 'Safari/537.36"', '"52.205.198.241"']


AttributeError: 'list' object has no attribute 'length'